In [ ]:
# Generate all comments and reactions

from github import Github as pg
#Error handling -- retries and timeout isnt handled by Github Exception handling
from github import RateLimitExceededException, BadCredentialsException, BadAttributeException, GithubException, UnknownObjectException, BadUserAgentException
import requests #retries and timeout Github Exception handling
import time #sleep the process for sometime

import csv 
from collections.abc import Iterable #To check if a datatype is iterable 
from pandas import *

MY_TOTAL_TOKENS = 8
access_token1 = <add_token_here> 
access_token2 = <add_token_here> 
access_token3 = <add_token_here> 
access_token4 = <add_token_here> 

access_token5 = <add_token_here> 
access_token6 = <add_token_here> 
access_token7 = <add_token_here> 
access_token8 = <add_token_here> 

# Set initial API token
api_token_queue = [access_token1,access_token2,access_token3,access_token4,access_token5,access_token6,access_token7,access_token8]
languages = [ "HCL", "TypeScript", "Rust", "Lua", "Go", "Shell", "Makefile", "C", "Kotlin", "Python"]

In [ ]:
token_i = 0
g = pg(api_token_queue[token_i], per_page=100, retry=20)
# Create a CSV file and write the headers 
for lang in languages:
    # reading CSV file
    print("Hey there, I'm working on language:",lang)
    data = read_csv("../Output/"+lang+".csv") 
    # converting column data to list
    repo_names = data['Repo Name'].tolist()
    with open("../Output/PRs/"+lang+".csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Number", "Title", "Body", "State", "Merged", "Created At", "Updated At", "Comments URL", "#Comments","#Reactions", "Comments", "Reactions", "Number_of_Files_changed", "Files_name", "Number_of_commits", "Commit_Messages", "Commits_Data"])
        for repo_name in repo_names:
            this_rate= str(g.get_rate_limit())
            this_rate= int(this_rate.split(",")[1].split("=")[1])
            if(this_rate < 1000):
                token_i =(token_i + 1) % MY_TOTAL_TOKENS
            while True:
                try:
                    g = pg(api_token_queue[token_i], per_page=100, retry=20)
                    print(f"Extracting PRs from {repo_name} repo using token #{token_i}")
                    repo = g.get_repo(repo_name)
                    # Fetch all PRs (both open and closed)
                    pulls = repo.get_pulls(state='all', sort='created', base='master')
                    # Create a CSV file and write the headers
                    # Write the pull data to the CSV file
                    for pull in pulls:
                        while True:
                            try:
                                # extracting comments(inc reactions) from the issues
                                pulls_comments = pull.get_comments()
                                cmnt_cnt = 0 #To store the number of comments done on this issue
                                rxn_cnt = 0 #To store teh number of reactions on the issues
                                pull_comments = []
                                pull_reactions = []
                                for comment in pulls_comments:
                                    pull_comments.append(comment.body)
                                    cmnt_cnt +=1
                                    this_cmnt_rxns = []
                                    any_rxn = 0 # To check if there are any reactions on this comment
                                    for reaction in comment.get_reactions():
                                        #if no reactions, then just append null or put comma and proceed.
                                        if reaction.content == []:
                                            this_cmnt_rxns.append('NA')
                                        else:
                                            this_cmnt_rxns.append(reaction.content)
                                            any_rxn +=1
                                            rxn_cnt +=1
                                    #if no reactions, then just append null or put comma and proceed.
                                    if any_rxn > 0:
                                        pull_reactions.append(this_cmnt_rxns)
                                    else: 
                                        pull_reactions.append('NA')

                                #Pull file related data
                                # extracting file changes made by a pull request and associated metadata and storing it in a dict
                                files = pull.get_files()
                                file_names = []
                                for file1 in files:
                                    file_names.append(file1.filename)
                                changed_files = {}
                                for file in files:
                                    changed_files.update({
                                        'Name': file.filename,
                                        'Line Additions': file.additions,
                                        'Line Deletions': file.deletions,
                                        'Patch': file.patch
                                    })
                                #extracting commits of the pull requests
                                pull_commits = []
                                pr_commits = pull.get_commits()
                                for commit in pr_commits:
                                    pull_commits.append(commit.commit.message)

                                #Write row into the csv
                                writer.writerow([pull.number, pull.title, pull.body, pull.state, pull.merged, pull.created_at, pull.updated_at, pull.comments_url, cmnt_cnt, rxn_cnt, pull_comments, pull_reactions, pull.changed_files, file_names, pull.commits, pull_commits, changed_files])
                            except RateLimitExceededException as e:
                                print(e.status)
                                print('Inside Loop: Rate Limit exceeded')
                                time.sleep(300) #5 minutes
                                continue
                            except BadCredentialsException as e:
                                print(e.status)
                                print("Inside Loop: Bad Credentials exception")
                                break
                            except UnknownObjectException as e:
                                print(e.status)
                                print("Inside Loop: The repository doesn't exist anymore")
                                break
                            except GithubException as e:
                                print(e.status)
                                print("Inside Loop: Github General Exception")
                                break
                            except requests.exceptions.ConnectionError as e:
                                print("Inside Loop: Retries limit exceeded")
                                print(str(e))
                                time.sleep(10)
                                continue
                            except requests.exceptions.Timeout as e:
                                print("Inside Loop: Timeout exception")
                                print(str(e))
                                time.sleep(10)
                                continue
                            break
                except RateLimitExceededException as e:
                    print(e.status)
                    print('Rate Limit exceeded')
                    time.sleep(300) #5 minutes
                    continue
                except BadCredentialsException as e:
                    print(e.status)
                    print("Bad Credentials exception")
                    break
                except UnknownObjectException as e:
                    print(e.status)
                    print("The repository doesn't exist anymore")
                    break
                except GithubException as e:
                    print(e.status)
                    print("Github General Exception")
                    break
                except requests.exceptions.ConnectionError as e:
                    print("Retries limit exceeded")
                    print(str(e))
                    time.sleep(10)
                    continue
                except requests.exceptions.Timeout as e:
                    print("Timeout exception")
                    print(str(e))
                    time.sleep(10)
                    continue
                break
    print("Hey there, I'm done with language:",lang)